In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import yfinance as yf
import keras 
import os
import tensorflow
import datetime 
import requests
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [19]:
class DataPrep():
    def __init__(self,ticker,daysago,split):
        self.ticker = ticker.upper()
        self.daysago = daysago
        self.split = split

        
        df = yf.Ticker(ticker.upper()).history(period='max')
        if df.shape[1] == 7:
            df = df.drop(columns=['Dividends','Stock Splits'])
        df = df.drop(columns=['Open','High','Low','Volume'])

        for i in range(daysago,-2,-1):
            df['{}daysago'.format(i)] = df['Close'].shift(i)
            df = df.copy()
        df = df.drop(columns=['Close'])
        self.df = df.dropna()

        
        #scales the data, splits into train and test
        self.sc = MinMaxScaler(feature_range=(0,1))
        self.split = int(len(df)*split)
        self.X_train = self.sc.fit_transform(self.df)[:,:self.daysago+1][:self.split]
        self.X_test = self.sc.fit_transform(self.df)[:,:self.daysago+1][self.split:]
        self.Y_train = self.sc.fit_transform(self.df)[:,-1][:self.split]
        self.Y_test = self.sc.fit_transform(self.df)[:,-1][self.split:]

e = DataPrep('aapl',100,0.9)
f = e.boxSC(['aapl','msft','googl'])
f

{'AAPL': MinMaxScaler(), 'MSFT': MinMaxScaler(), 'GOOGL': MinMaxScaler()}

In [23]:
def MultiDataPrep(listOfTickers,daysago,split):
    X_train = []
    X_test = []
    Y_train = []
    Y_test = []
    for ticker in listOfTickers:
        data = DataPrep(ticker,daysago,split)
        X_train.append(data.X_train)
        X_test.append(data.X_test)
        Y_train.append(data.Y_train)
        Y_test.append(data.Y_test)
    return X_train,X_test,Y_train,Y_test

def boxSC(listOfTickers,daysago,split):
    dict = {}
    for ticker in listOfTickers:
        print(ticker)
        dict[ticker.upper()]= DataPrep(ticker,daysago,split).sc
    return dict

#takes the name of a csv file and returns a list of tickers, with an optional blacklist
def ReadIndex(filename='SP500',blacklist=['BRK.B','BF.B','WRK','SOLV','GEV','SWKS']):
    df = pd.read_csv(f'{filename}.csv')
    ListOfTickers = df['Symbol'].tolist()
    for ticker in blacklist:
        ListOfTickers.remove(ticker)
    return ListOfTickers

boxSC(ReadIndex(),100,0.9)

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BLDR
BG
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CHRW
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUBB
HUM
HBAN
HI

{'MMM': MinMaxScaler(),
 'AOS': MinMaxScaler(),
 'ABT': MinMaxScaler(),
 'ABBV': MinMaxScaler(),
 'ACN': MinMaxScaler(),
 'ADBE': MinMaxScaler(),
 'AMD': MinMaxScaler(),
 'AES': MinMaxScaler(),
 'AFL': MinMaxScaler(),
 'A': MinMaxScaler(),
 'APD': MinMaxScaler(),
 'ABNB': MinMaxScaler(),
 'AKAM': MinMaxScaler(),
 'ALB': MinMaxScaler(),
 'ARE': MinMaxScaler(),
 'ALGN': MinMaxScaler(),
 'ALLE': MinMaxScaler(),
 'LNT': MinMaxScaler(),
 'ALL': MinMaxScaler(),
 'GOOGL': MinMaxScaler(),
 'GOOG': MinMaxScaler(),
 'MO': MinMaxScaler(),
 'AMZN': MinMaxScaler(),
 'AMCR': MinMaxScaler(),
 'AEE': MinMaxScaler(),
 'AAL': MinMaxScaler(),
 'AEP': MinMaxScaler(),
 'AXP': MinMaxScaler(),
 'AIG': MinMaxScaler(),
 'AMT': MinMaxScaler(),
 'AWK': MinMaxScaler(),
 'AMP': MinMaxScaler(),
 'AME': MinMaxScaler(),
 'AMGN': MinMaxScaler(),
 'APH': MinMaxScaler(),
 'ADI': MinMaxScaler(),
 'ANSS': MinMaxScaler(),
 'AON': MinMaxScaler(),
 'APA': MinMaxScaler(),
 'AAPL': MinMaxScaler(),
 'AMAT': MinMaxScaler(),
 'AP

In [10]:
#takes the full list of tickers and a SINGLE metric, returns a grouped object of the metric, ranked by quantiles
metricsIWannaTest = ['state', 'industryKey', 'sectorKey', 'fullTimeEmployees', 'companyOfficers', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'sharesShort', 'sharesShortPriorMonth', 'sharesShortPreviousMonthDate', 'dateShortInterest', 'sharesPercentSharesOut', 'heldPercentInsiders', 'heldPercentInstitutions', 'shortRatio', 'shortPercentOfFloat', 'impliedSharesOutstanding', 'bookValue', 'priceToBook', 'mostRecentQuarter', 'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'pegRatio', 'lastSplitFactor', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 'lastDividendValue', 'lastDividendDate', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'recommendationMean', 'numberOfAnalystOpinions', 'totalCash', 'totalCashPerShare', 'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 'returnOnAssets', 'returnOnEquity', 'freeCashflow', 'operatingCashflow', 'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins', 'trailingPegRatio']
def getIndexMetric(listOfTickers, metric):

    #initializes empty array, for metric info
    metricinfo = []

    #loops through tickers and appends the metric info to the array
    for ticker in listOfTickers:
        print(ticker)
        metricinfo.append(yf.Ticker(ticker).info[metric])
    
    #converts the array to a df, groups by quantiles
    metricinfo = pd.DataFrame(metricinfo,columns=[f'{metric}'],index=ReadIndex())
    metricinfo = metricinfo.groupby(pd.qcut(metricinfo[f'{metric}'], q=10),observed=False)
    return metricinfo
getIndexMetric(ReadIndex(), 'marketCap')


MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BLDR
BG
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CHRW
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUBB
HUM
HBAN
HI

In [ ]:
#takes the grouped object of a metric and returns a dictionary of train/test data for each quantile
def groupedIntoTrainableData(df_grouped):
    boxes = {}
    quant = 10
    
    #loops through the grouped object and compiles data for each quantile
    for group_name, group_data in df_grouped: 
        boxes[f'{quant}% box'] = DataPrep(group_data.index.tolist())
        quant += 10
        
    return boxes
groupedIntoTrainableData(getIndexMetric(ReadIndex(), 'marketCap'))